In [0]:
%py
'''
1° Cria catalogo bikestore em external location
2° Criar pastas no container e organização em workspace databricks
<storage-account>/<container>/
│
├── bikestore/
│   ├── resource/                 # dados de origem e entrega
│   │   ├── origem/               # arquivos CSV/Parquet recebidos da fonte
│   │   └── destino/              # exportações (CSV, Parquet, Excel) para clientes ou setores
│   │
│   ├── bronze/                   # dados crus (raw)
│   │
│   ├── silver/                   # dados limpos / tratados
│   │
│   ├── gold/                     # dados agregados / analytics

3° Criar Schema logistics  para receber as tabelas e volumes 
4° Criar Volume (bikestore_resource) com a Origem dos Dados (deletar todos anteriores para nao ter possivel conflito)
5° Criar Pipiline de dados completa com orquestração
'''

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bikestore.logistics;
create external volume if not exists bikestore.logistics.bikestore_resource 
comment 'Volume projeto bikestore'
location 'abfss://uc-ext-azure@externalazurenayara.dfs.core.windows.net/bikestore/resource'


In [0]:
display(dbutils.fs.ls('/Volumes/bikestore/logistics/bikestore_resource/origem/')) # map volume origem

In [0]:
display(dbutils.fs.ls('abfss://uc-ext-azure@externalazurenayara.dfs.core.windows.net/bikestore'))# map externo

In [0]:
# Definir pastas do projetos em variaveis para facilitar
bronze_path   = 'abfss://uc-ext-azure@externalazurenayara.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalazurenayara.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://uc-ext-azure@externalazurenayara.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://uc-ext-azure@externalazurenayara.dfs.core.windows.net/bikestore/resource/origem/'
resource_path_volume = '/Volumes/bikestore/logistics/bikestore_resource/origem/'


In [0]:
display(dbutils.fs.ls(bronze_path))# map variavel

In [0]:
df_brands=spark.read.csv(f'{resource_path}/brands.csv',
                         header=True,
                         inferSchema=True,
                         sep=','
                                              
                         )

In [0]:
display(df_brands)

In [0]:
#salvar em parquet como delta 
df_brands.write\
    .mode('overwrite')\
    .format('delta')\
    .option("mergeSchema", "true") \
    .save(f"{bronze_path}/brands")


In [0]:
%sql
--criar uma tabela com o external location 'nao iremos usar na bronze' pois sao os dados brutos e prescisam de tratamento (geralmente nao sao criadas tabelas pra esta parte , mas nao é uma regra)

CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_brands
LOCATION 'abfss://uc-ext-azure@externalazurenayara.dfs.core.windows.net/bikestore/bronze/brands/'

